In [1]:
import pandas as pd
import math
import numpy as np

In [4]:

df = pd.read_excel("/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/PDBbind_complete_general_set_exp.xlsx",header=1)

print(df.keys())

df_relevant = df[['PDB code','Subset','Affinity Data','pKd pKi pIC50']]

df_relevant

Index(['ID', 'PDB code', 'Subset', 'Complex Type', 'Resolution',
       'Affinity Data', 'pKd pKi pIC50', 'Release Year', 'Protein Name',
       'Ligand Name', 'Pubmed ID', 'Pubchem SID', 'EC number', 'Reference',
       'UniProt AC', 'Formula', 'Molecular Weight', 'Exact Mass',
       'No. of atoms', 'No. of bonds', 'Polar Surface Area', 'XLOGP3',
       'open banel LogP', 'HB donor', 'HB acceptor', 'Rotatable bonds',
       'Canonical SMILES'],
      dtype='object')


,PDB code,Subset,Affinity Data,pKd pKi pIC50
0,10gs,refined,Ki=0.4uM,6.40
1,11gs,general,Ki=1.5uM,5.82
2,13gs,general,Ki=24uM,4.62
3,16pk,general,Ki=6uM,5.22
4,184l,refined,Kd=19uM,4.72
...,...,...,...,...
19438,6uyx,general,Kd=1.0uM,6.00
19439,6uyy,general,Kd=1.9uM,5.72
19440,6uyz,general,Kd=4.4uM,5.36
19441,6v1c,general,Kd=65uM,4.19


In [5]:
a_list = []
b_list = []
for i in df_relevant["Affinity Data"]:
    if "=" in i:
        a,b = i.split("=")
    elif "~" in i:
        a,b = i.split("~")
    a_list.append(a)
    b_list.append(b)
    
df_relevant["Affinity Data Type"] = a_list
df_relevant["Affinity Data Value"] = b_list

value_list = []
for j in df_relevant["Affinity Data Value"]:
    if "uM" in j:
        j = j[:-2]
        value = float(j)*1000
    if "nM" in j:
        j = j[:-2]
        value = float(j)
    if "pM" in j:
        j = j[:-2]
        value = float(j)/1000
    value_list.append(value)
    
df_relevant["Affinity Data Value"] = value_list


dG_list = []
recipK_list = []
for u in df_relevant["Affinity Data Value"]:
    dG = 8.314*310*math.log(u)
    recipK = 1/float(u)
    dG_list.append(dG)
    recipK_list.append(recipK)

df_relevant["delta G"] = dG_list
df_relevant["1/K"] = recipK_list

df_refined = df_relevant[df_relevant["Subset"] == "refined"]
df_refined = df_refined[["PDB code","Affinity Data Type","Affinity Data Value","pKd pKi pIC50","delta G","1/K"]]
df_refined.reset_index(drop=True,inplace=True)
df_refined.to_csv("/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/experiments/refined_set.csv")
print(df_refined.shape)

df_general = df_relevant[df_relevant["Subset"] == "general"]
df_general = df_general[["PDB code","Affinity Data Type","Affinity Data Value","pKd pKi pIC50","delta G","1/K"]]
df_general.reset_index(drop=True,inplace=True)
df_general.to_csv("/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/experiments/general_set.csv")
print(df_general.shape)

(5316, 6)


/tmp/ipykernel_5114/3933448714.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant["Affinity Data Type"] = a_list
/tmp/ipykernel_5114/3933448714.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant["Affinity Data Value"] = b_list
/tmp/ipykernel_5114/3933448714.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

(14127, 6)


In [6]:

df = pd.read_excel("/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/PDBbind_complete_general_set_exp.xlsx",header=1)

print(df.keys())

df_ec = df[['PDB code','EC number']]

df_ec

ec_1 = []
ec_2 = []
ec_3 = []
ec_4 = []


not_unique = []
for n, j in enumerate(df_ec["EC number"]):
    b = j.split(";")
    if len(b)>1:
        not_unique.append(n)
        
df_ec.drop(not_unique,axis="index",inplace=True)
df_ec.reset_index(inplace=True,drop=True)
print(len(not_unique))

for i in df_ec["EC number"]:
    a = i.split(".")
    ec_1.append(a[2])
    try:
        ec_2.append(a[3])
    except:
        ec_2.append("-")
    try:
        ec_3.append(a[4])
    except:
        ec_3.append("-")
    try:
        ec_4.append(a[5])
    except:
        ec_4.append("-")
        
print(len(ec_1),len(ec_2),len(ec_3),len(ec_4))
df_ec["Class"] = ec_1
df_ec["Subclass"] = ec_2
df_ec["Subsubclass"] = ec_3


remove = []
for n, m in enumerate(df_ec["Class"]):
    try:
        tmp = int(m)
        if tmp > 10:
            remove.append(n)
    except:
        remove.append(n)
        
df_ec.drop(remove,axis="index",inplace=True)
df_ec.reset_index(inplace=True,drop=True)


remove = []
for n, m in enumerate(df_ec["Subclass"]):
    if m == "-":
        remove.append(n)
        
df_ec.drop(remove,axis="index",inplace=True)
df_ec.reset_index(inplace=True,drop=True)
df_ec.to_csv("/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/EC_numbers/ec_numbers_all.csv")

Index(['ID', 'PDB code', 'Subset', 'Complex Type', 'Resolution',
       'Affinity Data', 'pKd pKi pIC50', 'Release Year', 'Protein Name',
       'Ligand Name', 'Pubmed ID', 'Pubchem SID', 'EC number', 'Reference',
       'UniProt AC', 'Formula', 'Molecular Weight', 'Exact Mass',
       'No. of atoms', 'No. of bonds', 'Polar Surface Area', 'XLOGP3',
       'open banel LogP', 'HB donor', 'HB acceptor', 'Rotatable bonds',
       'Canonical SMILES'],
      dtype='object')
218
19225 19225 19225 19225


/tmp/ipykernel_5114/2411636094.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ec.drop(not_unique,axis="index",inplace=True)
/tmp/ipykernel_5114/2411636094.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ec["Class"] = ec_1
/tmp/ipykernel_5114/2411636094.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

In [10]:
unique_classes = []
for i in df_ec["Class"]:
    if int(i) in unique_classes:
        continue
    else:
        unique_classes.append(int(i))
unique_classes.sort()
print("Classes",unique_classes)

unique_subclasses = []
for i in df_ec["Subclass"]:
    if int(i) in unique_subclasses:
        continue
    else:
        unique_subclasses.append(int(i))
unique_subclasses.sort()
print("Subclasses",unique_subclasses)

unique_subsubclasses = []
for i in df_ec["Subsubclass"]:
    try:
        if int(i) in unique_subsubclasses:
            continue
        else:
            unique_subsubclasses.append(int(i))
    except:
        continue
unique_subsubclasses.sort()
print("Subsubclasses",unique_subsubclasses)


for c in unique_classes:
    df_tmp = df_ec.loc[df_ec["Class"] == str(c)]
    df_tmp.to_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/EC_numbers/{c}_x_x_x.csv")
    print(f"Dataframe for class {c} has shape:",df_tmp.shape)

for c in unique_classes:
    for s in unique_subclasses:
        df_tmp = df_ec.loc[df_ec["Class"] == str(c)]
        df_tmp = df_tmp[df_tmp["Subclass"] == str(s)]
        if df_tmp.shape[0] == 0:
            continue
        else:
            df_tmp.to_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/EC_numbers/{c}_{s}_x_x.csv")
            print(f"Dataframe for class {c} and subclass {s} has shape:",df_tmp.shape)

for c in unique_classes:
    for sc in unique_subclasses:
        for ssc in unique_subsubclasses:
            df_tmp = df_ec.loc[df_ec["Class"] == str(c)]
            df_tmp = df_tmp[df_tmp["Subclass"] == str(sc)]
            df_tmp = df_tmp[df_tmp["Subsubclass"] == str(ssc)]
            if df_tmp.shape[0] == 0:
                continue
            else:
                df_tmp.to_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/EC_numbers/{c}_{sc}_{ssc}_x.csv")
                print(f"Dataframe for class {c}, subclass {sc} and subsubclass {ssc} has shape:",df_tmp.shape)


Classes [1, 2, 3, 4, 5, 6, 7]
Subclasses [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 20, 99]
Subsubclasses [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 98, 99]
Dataframe for class 1 has shape: (460, 5)
Dataframe for class 2 has shape: (4634, 5)
Dataframe for class 3 has shape: (4812, 5)
Dataframe for class 4 has shape: (762, 5)
Dataframe for class 5 has shape: (295, 5)
Dataframe for class 6 has shape: (423, 5)
Dataframe for class 7 has shape: (1, 5)
Dataframe for class 1 and subclass 1 has shape: (125, 5)
Dataframe for class 1 and subclass 2 has shape: (33, 5)
Dataframe for class 1 and subclass 3 has shape: (13, 5)
Dataframe for class 1 and subclass 4 has shape: (14, 5)
Dataframe for class 1 and subclass 5 has shape: (44, 5)
Dataframe for class 1 and subclass 6 has shape: (8, 5)
Dataframe for class 1 and subclass 7 has shape: (2, 5)
Dataframe for class 1 and subclass 8 has shape: (6, 5)
Dataframe for class 1 and subclass 1

In [15]:
df = pd.read_csv("/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/results/feature_selection_RFECV_results.csv",index_col=0)

des_type_list = []
estimator_list = []

for i in df["add. information"]:
    des_type,tmp,modeltype = i.split(" ")
    des_type_list.append(des_type)
    estimator_list.append(modeltype)

df["Descriptortype"] = des_type_list
df["Estimator"] = estimator_list

df.drop("add. information",axis=1,inplace=True)
df.to_csv("/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/results/feature_selection_RFECV_results.csv")


In [28]:
descriptors = ["GAP","slim"]
Classes = ["1","2","3","4","5","6","7"]

for des in descriptors:
    test_grail = f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/GRAIL-scores/PDBbind_general_set_grail_scores_{des}.csv"
    val_grail = f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/GRAIL-scores/PDBbind_core_set_grail_scores_{des}.csv"
    test_exp = f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/experiments/general_set.csv"
    val_exp = f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/experiments/core_set.csv"

    df_test_grail = pd.read_csv(test_grail)
    df_val_grail = pd.read_csv(val_grail)
    df_test_exp = pd.read_csv(test_exp)
    df_val_exp = pd.read_csv(val_exp)
    for c in Classes:
        df_class = pd.read_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/EC_numbers/{c}_x_x_x.csv",index_col=0)
        PDBs = df_class["PDB code"]
        df_test_grail_class = df_test_grail[df_test_grail["PDB code"].isin(PDBs)]
        df_val_grail_class = df_val_grail[df_val_grail["PDB code"].isin(PDBs)]
        df_test_exp_class = df_test_exp[df_test_exp["PDB code"].isin(PDBs)]
        df_val_exp_class = df_val_exp[df_val_exp["PDB code"].isin(PDBs)]
        df_test_grail_class.to_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/GRAIL-scores/PDBbind_general_set_grail_scores_{des}_class{c}.csv",index=False)
        df_val_grail_class.to_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/GRAIL-scores/PDBbind_core_set_grail_scores_{des}_class{c}.csv",index=False)
        df_test_exp_class.to_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/experiments/general_set_class{c}.csv",index=False)
        df_val_exp_class.to_csv(f"/data/shared/projects/pharmacophore_hot_spot_analysis/phantomdragon/data/experiments/core_set_class{c}.csv",index=False)
        print(f"Class {c} done")

Class 1 done
Class 2 done
Class 3 done
Class 4 done
Class 5 done
Class 6 done
Class 7 done
Class 1 done
Class 2 done
Class 3 done
Class 4 done
Class 5 done
Class 6 done
Class 7 done
